In [1]:
import os
import re
from copy import deepcopy
from pathlib import Path

import cheetah.bmad

In [2]:
os.environ["LCLS_LATTICE"] = str((Path(".").absolute().parent / "lcls-lattice"))
os.environ["LCLS_LATTICE"]

'/Users/jankaiser/Documents/DESY/lcls-lattice'

In [3]:
lattice_file_path = (
    Path("$LCLS_LATTICE") / "bmad" / "models" / "cu_hxr" / "cu_hxr.lat.bmad"
)
lattice_file_path

PosixPath('$LCLS_LATTICE/bmad/models/cu_hxr/cu_hxr.lat.bmad')

In [4]:
resolved_lattice_file_path = Path(
    *[
        os.environ[part[1:]] if part.startswith("$") else part
        for part in lattice_file_path.parts
    ]
)
resolved_lattice_file_path

PosixPath('/Users/jankaiser/Documents/DESY/lcls-lattice/bmad/models/cu_hxr/cu_hxr.lat.bmad')

In [5]:
lines = cheetah.bmad.read_clean_lines(resolved_lattice_file_path)
lines[:20]

['beginning[beta_a] =  5.91253676811640894e+000',
 'beginning[alpha_a] =  3.55631307633660354e+000',
 'beginning[beta_b] =  5.91253676811640982e+000',
 'beginning[alpha_b] =  3.55631307633660398e+000',
 'beginning[e_tot] = 6e6',
 'parameter[geometry] = open',
 'parameter[particle] = electron',
 'beginning[theta_position] = -35*pi/180',
 'beginning[z_position] = 3050.512000 - 1032.60052',
 'beginning[x_position] = 10.44893',
 'setsp = 0',
 'setcus = 0',
 'setal = 0',
 'setda = 0',
 'setxleap2 = 0',
 'sethxrss = 0',
 'setsxrss = 0',
 'setcbxfel = 0',
 'setpepx = 0',
 'intgsx = 30.0']

In [6]:
merged_lines = cheetah.bmad.merge_delimitter_continued_lines(
    lines, delimitter="&", remove_delimitter=True
)
merged_lines = cheetah.bmad.merge_delimitter_continued_lines(
    merged_lines, delimitter=",", remove_delimitter=False
)
merged_lines = cheetah.bmad.merge_delimitter_continued_lines(
    merged_lines, delimitter="{", remove_delimitter=False
)
len(lines), len(merged_lines)

(14409, 12215)

In [7]:
property_assignment_pattern = r"[a-z0-9_]+\[[a-z0-9_%]+\]\s*=.*"
variable_assignment_pattern = r"[a-z0-9_]+\s*=.*"
element_definition_pattern = r"[a-z0-9_]+\s*\:.*"
line_definition_pattern = r"[a-z0-9_]+\s*\:\s*line\s*=\s*\(.*\)"
use_line_pattern = r"use\s*\,\s*[a-z0-9_]+"

num_successful = 0
num_property_assignment = 0
num_variable_assignment = 0
num_element_definition = 0
num_line_definition = 0
num_use_line = 0
for line in merged_lines:
    if re.fullmatch(property_assignment_pattern, line):
        num_successful += 1
        num_property_assignment += 1
    elif re.fullmatch(variable_assignment_pattern, line):
        num_successful += 1
        num_variable_assignment += 1
    elif re.fullmatch(element_definition_pattern, line):
        num_successful += 1
        num_element_definition += 1
    elif re.fullmatch(line_definition_pattern, line):
        num_successful += 1
        num_line_definition += 1
    elif re.fullmatch(use_line_pattern, line):
        num_successful += 1
        num_use_line += 1
    else:
        print(line)
        break

print("")
print("######################################")
print(f"num_successful: {num_successful} / {len(merged_lines)}")
print("--------------------------------------")
print(f"{num_property_assignment = }")
print(f"{num_variable_assignment = }")
print(f"{num_element_definition = }")
print(f"{num_line_definition = }")
print(f"{num_use_line = }")
print("######################################")


######################################
num_successful: 12215 / 12215
--------------------------------------
num_property_assignment = 4256
num_variable_assignment = 1684
num_element_definition = 6274
num_line_definition = 0
num_use_line = 1
######################################


In [8]:
def assign_property(line: str, context: dict) -> dict:
    """Assign a property to the context."""
    pattern = r"([a-z0-9_]+)\[([a-z0-9_%]+)\]\s*=(.*)"
    match = re.fullmatch(pattern, line)

    object_name = match.group(1).strip()
    property_name = match.group(2).strip()
    property_expression = match.group(3).strip()  # TODO: Evaluate expression first

    if object_name not in context:
        context[object_name] = {}
    context[object_name][property_name] = property_expression

    return context

In [9]:
def assign_variable(line: str, context: dict) -> dict:
    """Assign a variable to the context."""
    pattern = r"([a-z0-9_]+)\s*=(.*)"
    match = re.fullmatch(pattern, line)

    variable_name = match.group(1).strip()
    variable_expression = match.group(2).strip()  # TODO: Evaluate expression first

    context[variable_name] = variable_expression

    return context

In [13]:
def define_element(line: str, context: dict) -> dict:
    """Define an element in the context."""
    pattern = r"([a-z0-9_]+)\s*\:\s*([a-z0-9_]+)(\,(.*))?"
    match = re.fullmatch(pattern, line)

    print(line)

    element_name = match.group(1).strip()
    element_type = match.group(2).strip()
    element_properties = {"element_type": element_type}

    if match.group(3) is not None:
        element_properties_string = match.group(4).strip()

        property_pattern = r"([a-z0-9_]+\s*\=\s*\"[^\"]+\"|[a-z0-9]+\s*\=\s*[^\=\,\"]+)"
        property_matches = re.findall(property_pattern, element_properties_string)

        for property_string in property_matches:
            property_string = property_string.strip()

            property_name, property_expression = property_string.split("=")
            property_name = property_name.strip()
            property_expression = property_expression.strip()

            element_properties[property_name] = property_expression

    context[element_name] = element_properties

    return context

In [14]:
def define_line(line: str, context: dict) -> dict:
    """Define a beam line in the context."""
    pattern = r"([a-z0-9_]+)\s*\:\s*line\s*=\s*\((.*)\)"
    match = re.fullmatch(pattern, line)

    line_name = match.group(1).strip()
    line_elements_string = match.group(2).strip()

    line_elements = []
    for element_name in line_elements_string.split(","):
        element_name = element_name.strip()

        line_elements.append(element_name)

    context[line_name] = line_elements

    return context

In [15]:
context = {}
for line in merged_lines:
    if re.fullmatch(property_assignment_pattern, line):
        context = assign_property(line, context)
    elif re.fullmatch(variable_assignment_pattern, line):
        context = assign_variable(line, context)
    elif re.fullmatch(line_definition_pattern, line):
        context = define_line(line, context)
    elif re.fullmatch(element_definition_pattern, line):
        context = define_element(line, context)
    elif re.fullmatch(use_line_pattern, line):
        print(line)
        break

dbmark80: marker
dbmark81: marker
dbmark97: marker
dbmark82: marker
dbmark98: marker
dbmark83: marker
dbmark28: marker
dbmark29: marker
wooddoor: marker
beggunb: marker
endgunb: marker
begl0b: marker
endl0b: marker
beghtr: marker
endhtr: marker
begdiag0: marker
enddiag0: marker
begcol0: marker
endcol0: marker
begl1b: marker
endl1b: marker
begbc1b: marker
endbc1b: marker
begcol1: marker
endcol1: marker
begl2b: marker
endl2b: marker
begbc2b: marker
endbc2b: marker
begemit2: marker
endemit2: marker
begl3b: marker
endl3b: marker
begext: marker
endext: marker
begdog: marker
enddog: marker
begbyp: marker
endbyp: marker
begsph: marker
endsph: marker
begslth: marker
endslth: marker
begspd_1: marker
endspd_1: marker
begspd_2: marker
endspd_2: marker
begspd_3: marker
endspd_3: marker
begsltd: marker
endsltd: marker
begsps: marker
endsps: marker
begslts: marker
endslts: marker
begbsys: marker
endbsys: marker
begltus: marker
endltus: marker
begunds: marker
endunds: marker
begdmps_1: marker
enddmps

In [16]:
context

{'beginning': {'beta_a': '5.91253676811640894e+000',
  'alpha_a': '3.55631307633660354e+000',
  'beta_b': '5.91253676811640982e+000',
  'alpha_b': '3.55631307633660398e+000',
  'e_tot': '6e6',
  'theta_position': '-35*pi/180',
  'z_position': '3050.512000 - 1032.60052',
  'x_position': '10.44893'},
 'parameter': {'geometry': 'open', 'particle': 'electron'},
 'setsp': '0',
 'setcus': '0',
 'setal': '0',
 'setda': '0',
 'setxleap2': '0',
 'sethxrss': '0',
 'setsxrss': '0',
 'setcbxfel': '0',
 'setpepx': '0',
 'intgsx': '30.0',
 'intghx': '30.0',
 'cb': '1.0e10/c_light',
 'gev2mev': '1000.0',
 'in2m': '0.0254',
 'mc2': '510.99906e-6',
 'injdeg': '-35.0',
 'zoffinj': '0.012100',
 'xoff': '-25.610*in2m',
 'yoff': '25.570*in2m',
 'emitxn': '1.00e-06',
 'emityn': '1.00e-06',
 'bleng': '0.83e-03',
 'esprd': '2.00e-05',
 'e00': '0.006',
 'e0i': '0.064',
 'ei': '0.135',
 'ebc1': '0.220',
 'ebc2': '5.000',
 'ef': '8.000',
 'eu': '8.000',
 'brho00': 'cb*e00',
 'brho0i': 'cb*e0i',
 'brhoi': 'cb*ei'

In [ ]:
line = 'ssp1h: pipe, l = 0.05, type ="@1,1.38s3.00", descrip = "deferred sextupole"'
line

'ssp1h: pipe, l = 0.05, type ="@1,1.38s3.00", descrip = "deferred sextupole"'

In [ ]:
pattern = r"([a-z0-9_]+)\s*"
match = re.fullmatch(pattern, line)
match[3]

'  5.91253676811640894e+000'